# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk), Dana Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [1]:
from nord_h2ub_hexhex import *

Custom functions imported successfully.


### Base Parameters:

In [2]:
# Example usage
combined_tabs, dropdowns = create_combined_dropdowns_tabs()

In [4]:
#get the values after setting the inputs by the user
values = get_dropdown_values(dropdowns)

#Calculate remaining values based on inputs
calc_values = compute_other_values(values)

### Dictionary:

In [ ]:
base_parameters = {
    'year': values['year'],
    'start_date': calc_values['start_date'],
    'end_date': calc_values['end_date'],
    'area': values['price_zone'],
    'product': values['product'],
    'scenario': [values['base_scen']] + values['other_scen'].split('\n'),
    'frequency': values['frequency'],
    'model_name': values['model_name'],
    'temporal_block': calc_values['temporal_block'],
    'stochastic_scenario': values['stoch_scen'],
    'stochastic_structure': values['stoch_struc'],
    'report_name': values['report_name'],
    'reports': values['outputs'],
    'electrolyzer_type': values['electrolysis'],
    'des_segments_electrolyzer': values['des_segments_electrolyzer'],
    'share_of_dh_price_cap': values['share_of_dh_price_cap'],
    'price_level_power': values['number_price_level_power'],
    'power_price_variance': values['power_price_variance'],
    'roll_forward_use': values['roll_forward'],
    'roll_forward_size': calc_values['roll_forward_size'],
    'num_slices': values['num_slices'],
    'num_steps': calc_values['num_steps']
}

### Execute standard routine:

In [ ]:
pm.execute_notebook(
    'data_preparation_nord_h2ub_spine.ipynb',
    'please_ignore.ipynb',
    parameters=base_parameters
)

In [1]:
exit()

In [1]:
from time import sleep
from ipylab import JupyterFrontEnd

In [4]:
def save_terminate():
    # Save the notebook
    from notebook import list_running_servers
    import nbformat
    
    for server in list_running_servers():
        url = server['url']
        token = server['token']
        notebook_file_path = notebookapp.get_current_notebook_path()
        with open(notebook_file_path, 'r') as file:
            nb = nbformat.read(file, as_version=4)
        nbformat.write(nb, notebook_file_path)

    print("Notebook saved and will terminate Jupyter soon.")
    
    # Optionally, if you want to close the browser tab automatically
    # from IPython.display import display, Javascript
    # display(Javascript('window.close()'))
    
    time.sleep(5)
    
    # Shutdown the Jupyter server
    server_url = os.getenv('JUPYTER_SERVER_URL')  # Ensure this environment variable is set
    if server_url:
        response = requests.post(f'{server_url}/api/shutdown', headers={
            'Authorization': f'token {os.getenv("JUPYTER_TOKEN")}'
        })
        if response.status_code == 200:
            print("Jupyter server terminated successfully.")
        else:
            print(f"Failed to terminate Jupyter server: {response.status_code}")

In [5]:
save_terminate()

ImportError: cannot import name 'list_running_servers' from 'notebook' (C:\Users\jfg.eco\AppData\Local\anaconda3\envs\Spineopt_py309\lib\site-packages\notebook\__init__.py)

In [ ]:
import os
import time
import requests
from IPython.display import display, Javascript
import subprocess
import platform

def save_and_shutdown():
    # Save the notebook
    print("Saving the notebook...")
    try:
        # Execute JavaScript to save the notebook
        display(Javascript('IPython.notebook.save_checkpoint()'))
        print("Notebook save requested.")
    except Exception as e:
        print(f"Error saving notebook: {e}")

    # Optionally, close the browser tab
    try:
        display(Javascript('window.close()'))
        print("Attempted to close the browser tab.")
    except Exception as e:
        print(f"Error closing browser tab: {e}")

    # Pause to ensure saving and closing actions complete
    time.sleep(5)

    # Shutdown the Jupyter server
    print("Shutting down the Jupyter server...")
    try:
        # Get the server URL and token from environment variables
        server_url = os.getenv('JUPYTER_SERVER_URL', 'http://localhost:8888')
        token = os.getenv('JUPYTER_TOKEN', 'your_token_here')
        
        # Send a request to shutdown the server
        response = requests.post(f'{server_url}/api/shutdown', headers={
            'Authorization': f'token {token}'
        })
        
        if response.status_code == 200:
            print("Jupyter server shutdown request sent successfully.")
        else:
            print(f"Failed to send shutdown request. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error sending shutdown request: {e}")

    # Ensure Jupyter server process is terminated
    try:
        if platform.system() == "Windows":
            subprocess.run(["taskkill", "/F", "/IM", "jupyter-notebook.exe"], check=True)
        else:
            subprocess.run(["pkill", "-f", "jupyter-notebook"], check=True)
        print("Jupyter server process terminated.")
    except Exception as e:
        print(f"Error terminating Jupyter server process: {e}")

# Call the function
save_and_shutdown()
